# Ensemble Learning Using Random Forests
This a lab session to use tree ensembles, in particular Random Forests, to build a interesting classifier related to human activity recognition using mobile phone data.

## Read Documentation about the Data Sources

The features selected for this UCI database come from the accelerometer and gyroscope 3-axial raw signals tAcc-XYZ and tGyro-XYZ. These time domain signals (prefix 't' to denote time) were captured at a constant rate of 50 Hz. Then they were filtered using a median filter and a 3rd order low pass Butterworth filter with a corner frequency of 20 Hz to remove noise. Similarly, the acceleration signal was then separated into body and gravity acceleration signals (tBodyAcc-XYZ and tGravityAcc-XYZ) using another low pass Butterworth filter with a corner frequency of 0.3 Hz. 

Subsequently, the body linear acceleration and angular velocity were derived in time to obtain Jerk signals (tBodyAccJerk-XYZ and tBodyGyroJerk-XYZ). Also the magnitude of these three-dimensional signals were calculated using the Euclidean norm (tBodyAccMag, tGravityAccMag, tBodyAccJerkMag, tBodyGyroMag, tBodyGyroJerkMag). 

Finally a Fast Fourier Transform (FFT) was applied to some of these signals producing fBodyAcc-XYZ, fBodyAccJerk-XYZ, fBodyGyro-XYZ, fBodyAccJerkMag, fBodyGyroMag, fBodyGyroJerkMag. (Note the 'f' to indicate frequency domain signals). 

These signals were used to estimate variables of the feature vector for each pattern:  
'-XYZ' is used to denote 3-axial signals in the X, Y and Z directions.

tBodyAcc-XYZ
tGravityAcc-XYZ
tBodyAccJerk-XYZ
tBodyGyro-XYZ
tBodyGyroJerk-XYZ
tBodyAccMag
tGravityAccMag
tBodyAccJerkMag
tBodyGyroMag
tBodyGyroJerkMag
fBodyAcc-XYZ
fBodyAccJerk-XYZ
fBodyGyro-XYZ
fBodyAccMag
fBodyAccJerkMag
fBodyGyroMag
fBodyGyroJerkMag

The set of variables that were estimated from these signals are: 

mean(): Mean value
std(): Standard deviation
mad(): Median absolute deviation 
max(): Largest value in array
min(): Smallest value in array
sma(): Signal magnitude area
energy(): Energy measure. Sum of the squares divided by the number of values. 
iqr(): Interquartile range 
entropy(): Signal entropy
arCoeff(): Autorregresion coefficients with Burg order equal to 4
correlation(): correlation coefficient between two signals
maxInds(): index of the frequency component with largest magnitude
meanFreq(): Weighted average of the frequency components to obtain a mean frequency
skewness(): skewness of the frequency domain signal 
kurtosis(): kurtosis of the frequency domain signal 
bandsEnergy(): Energy of a frequency interval within the 64 bins of the FFT of each window.
angle(): Angle between to vectors.

Additional vectors obtained by averaging the signals in a signal window sample. These are used on the angle() variable:

gravityMean
tBodyAccMean
tBodyAccJerkMean
tBodyGyroMean
tBodyGyroJerkMean

## Import the Data and Browse
- Data is located in <project_root>/exercises/data/samsungdata.csv
    - For solutions notebook, the relative file path is '../data/samsungdata.csv'
    - For exercise notebook, the relative path is './data/samsungdata.csv'


In [ ]:
%pylab inline
import seaborn as sbn
import pandas as pd
X_raw = pd.read_csv('https://github.com/gte620v/PracticalDataScience/raw/master/exercises/data/samsungdata.csv')

### First, take a look at the data and get acquainted
For example, you could do things like
```python
X_raw.shape
```

```python
X_raw.head()
```

For a more detailed overview of your data, look at
```python
X_raw.describe()
```

In particular, browse to see if there are any variables in your data that are NOT numerical sensor measurements to be used for prediction.

### Cleaning up the data and getting ready for Machine Learning
We'll do a very crude data cleaning step, just enough to get the data in a usable form.

There are two columns, "Unnamed: 0", "subject" and "activity" that are categorical and/or not useful.

1. The 'activity' column contains the targets to be used for classification (activity categories). Extract that into a separate variable.
Hint:
```python
truth_har = X_raw['activity']
```
Take a look at the distribution of the activity class labels.

2. Do a similar analysis for the 'subject' column.

3. Remove the 3 columns 'Unnamed: 0', 'subject', and 'activity' from X_raw

## Build a RF Classifier as a Black Box
Some observations about this problem
- What if we don't know much about Human Activity Recognition?
- What if we don't know a lot about the sensors (accelerometers, gyroscopes, etc)?
- Can we get some results very fast that will give us reasonably good performance?

Make a pass at this problem using Random Forest Classifiers. Pick some reasonable defaults and test your algorithm.
```python
import sklearn.ensemble as ens # check out: ens.RandomForestClassifier
from sklearn.model_selection import train_test_split
```

Use the sklearn helper function 'train_test_split' to split your matrix/data into test and training set. Set aside 20% of the data for testing final model performance.

Select some reasonable parameter choices for your RandomForestClassifier.

For example,
```python
rf_classifier = ens.RandomForestClassifier(max_features=<choose one>, n_estimators=<choose>, oob_score=True)
```

Note the following parameters are key to exploring the bias/variance tradeoff of Random Forests.
- max_features : number of features to be randomly selected in the construction of individual trees
    - Recommended to be approx $\sqrt{N_{features}}$
- n_estimators : total number of trees to be built for the ensemble. Pick some value between 25 and 500


Fit the model to the training data, and calculate the OOB error/accuracy

Now, calculate the test set error for this model.
- How is the performance?
- How close was the out-of-bag error to the test set error? Any insights to gain there?

Hint: you can make predictions on the test set like this:
```python
m = rf.fit(X_train,y_train)
y_test_predict = m.predict(X_test)
```

## RF Classifier Hyper-parameter Tuning
We were able to achieve reasonably good performance, with no application of domain knowledge, in the previous example. This is an impressive feat. However, how can we really know if this performance is good? We only arbitrarily selected a single parameter setting?

Random Forests have a variety of parameters that can be tuned before training the model. These parameters fall into a couple categories:
1. Changing the loss functions used to construct individual decision trees (we will not consider this)
2. Modifying the construction process of the Ensemble Learner.

As we learned in the lecture on Ensemble Learning and Random Forests, the trick to high performance is
1. Having the ability to create a large number of trees (wisdom of the crowds)
2. Ensuring each tree has 'novelty', in the sense of statistical independence from the other trees.
3. Making each tree satisfy the "Weak Learner" condition for convergence
3. Note 1 and 2 above are in conflict with each other. The more trees we construct, the hard it is to maintain statistical independence from the previous trees.

The two Random Forest hyper-parameters that align with our wishes are:
- max_features
    - Proper subset of features to be sampled in construction of the tree.
    - By having this number be much less than the total number of features, this introduces novelty and statistical independence between trees in the ensemble.
- n_estimators
    - The number of trees to construct.
    
Before we can begin a grid search, we need to get a sense for how much computational power we have behind the scenes to train a single model. This will help us estimate the time we need to complete a grid search.

As a suggestion, use max_features=25 and n_estimators=500 to get a ballpark estimate.


```python
%%time
rf_benchmark1 = ens.RandomForestClassifier(max_features=25, n_estimators=500, oob_score=True)
rf_benchmark1.fit(X_train,y_train)
```

**FIGURE OUT HOW MANY PARAMETER COMBINATIONS YOU WILL BE ABLE TO EXPLORE IN APPROX 3min OF COMPUTATION TIME**

One of the limiting performance factors in the previous example is that all the computations were run on a single CPU. To see this, open up your shell, and run htop in a tmux session. Then, rerun the above code segment and which the CPU usage and load.
```bash
user@host:~/$ htop
```
We can check the number of cores on the machine that's hosting our notebook, along with the specs of those processors by running the following shell command in our notebook:
```
!cat /proc/cpuinfo
```

From this, we can see colaboratory runs on 2 vCPUs.

Let's modify training to use ALL CPUs, and benchmark the progress. We do this by adding the parameter "n_jobs" when we construct the Random Forest (specifying the number of cores to use in training). If n_jobs=-1, all available cores are used in model training.
```python
%%time
rf_benchmark2 = ens.RandomForestClassifier(max_features=25, n_estimators=500, oob_score=True, n_jobs=-1)
rf_benchmark2.fit(X_train,y_train)
```

Based on the measured performance, create a grid search to find an 'optimal' mode over a variety of parameter combinations. Here's a suggestion for how this may look:
```python
from sklearn.model_selection import GridSearchCV
model_template = ens.RandomForestClassifier(random_state=123)
param_grid = { 
    'n_estimators': [<fill in here>],
    'max_features': [<fill in here>],
}
rf_gridCV = GridSearchCV(estimator=model_template,param_grid=param_grid,n_jobs=-1,cv=2)
rf_gridCV.fit(X_train,y_train)
rf_gridCV.best_params_
rf_gridCV.best_score_
```

Finally, evaluate the 'best' Random Forest on the test set.
```python
yp = rf_gridCV.best_estimator_.predict(X_test)
accuracy_score(y_pred=yp,y_true=y_test)
```

## Turbo-Charging RF Classifier Hyper-parameter Tuning
Now we will see the power of elastic computing resources in their ability to help us train a high performing model with minimal code changes. We'll make 3 changes to our hyper-parameter tuning grid search.
1. We'll stop our Google Compute instance, and **edit the specs of the machine to be MUCH more powerful**
    - We'll be able to restart our instance without needing to re-run our entire setup process. POWERFUL productivity booster!
2. We'll add-in 5-fold cross validation (CV) into the model selection/grid search process. 
    - CV turns out to be a better estimator of validation set error than out-of-bag error. OOB error is a neat computational trick that approximates test set error. However, it's a biases estimator and can lead to overfitting the model. We are going for RAW HORSEPOWER AND PERFORMANCE here.
3. We'll expand our grid search to try a larger variety of model topologies.

```python
model_template = ens.RandomForestClassifier(random_state=321)
param_grid = { 
    'n_estimators': [<fill in here>],
    'max_features': [<fill in here>],
}
rf_gridCV = GridSearchCV(estimator=model_template,param_grid=param_grid,n_jobs=-1,cv=5)
rf_gridCV.fit(X_train,y_train)
rf_gridCV.best_params_
rf_gridCV.best_score_
```
